## Analytic center cutting-plane method implementation
## Last updated: 22 September 2016 

## Introduction

## The computing environment

In [39]:
import numpy as np
import scipy.optimize as opt

## Computing the analytic center

In [34]:
def log_barrier(x, A, b):
    """
    Returns the logarithmic barrier associated with the set of 
    inequalities Ax <= b evaluated at x.
    
    It is preferable x, A, b are of type ndarray.
    """
    return -np.log(np.prod(b-np.dot(A, x)))

def logb_grad(x, A, b):
    """
    Returns the gradient of the logarithmic barrier associated 
    with the set of inequalities Ax <= b evaluated at x.
    """
    d = 1./(b-Ax)
    return np.dot(np.transpose(A), d)

def logb_hess(x, A, b):
    """
    Returns the Hessian of the logarithmic barrier associated 
    with the set of inequalities Ax <= b evaluated at x.
    """
    d = 1./(b-Ax)
    diagd =  np.diag(d)
    return np.dot(np.transpose(A), 
                  np.dot(np.linalg.matrix_power(diagd, 2), A))

In [12]:
def phase_i_func(z, t, A, b):
    (x, s) = np.vsplit(z, [z.shape[0]-1])
    return t*s - np.log(np.prod(s + (b-np.dot(A, x))))

def phase_i_grad(z, t, A, b):
    d = 1./(s + (b-np.dot(A, x)))
    return np.vstack(np.dot(np.transpose(A), d), 
                     (t + np.sum(d))) 

In [8]:
def phase_i_opt(A, b, t, mu, maxiter):
    """
    Returns a point x that satisfies Ax<b. 
    
    This is computed using the barrier method which relies on
    Newton's method.
    """
    x = np.zeros(b.shape)
    s = 0.1*np.fabs(np.amax(-b)) + np.amax(-b)
    z = np.vstack((x, s))
    i = 0
    while i < maxiter and z[-1] >= 0:
        result = minimize(phase_i_func, z, args = (t, A, b),
                          method='BFGS', jac=phase_i_grad,)
        z = result.x
        t = mu*t
    return z

In [10]:
def analytic_center(A, b, epsilon, t, mu, maxiter):
    """
    Returns the analytic center of the given polyhedron.
    
    polyhed : 2-tuple with ndarray elements 
        If  polyhed = { z | Az <= b }, for a matrix A and vector
        b, then polyhed = (A, b) where A and b are ndarray
        representations.
    """
    z = phase_i_opt(A, b, t, mu, maxiter)
    result = minimize(log_barrier, z, args=(A, b), 
                      method='Newton-CG', 
                      jac=logb_grad, hess = logb_hess)
    ac = result.x
    return ac

## Stopping criterion

In [ ]:

def dual_func

def lower_bound(multipliers, differences):
    

## Oracle

In [22]:
def feasible(x, constr, epsilon):
    for i in range(len(constr)):
        fi_x = constr[i](x)
        if fi_x > 0:
            return (False, fi_x, 
                    opt.approx_fprime(x, constr[i], epsilon))
        else: 
            return (True, )
        
def oracle(x, func, grad, constr, epsilon):
    feasibility = feasible(x, constr, epsilon)[0]
    if feasibility == False:
        fi_x = feasibility[1]
        grad_fi_x = feasibility[2]
        (a, b) = (feasibility[2], 
                  np.dot(grad_fi_x, x) - fi_x)
    else:
        (a, b) = (grad(x), np.dot(grad(x), x))
    return (feasibility, (a, b))

## The analytic center cutting method

In [23]:
def accpm(func, constr, epsilon, A, b, maxiter):
    """
    Solves the specified inequality constrained convex optimization 
    problem or feasbility problem via the analytic center cutting
    plane method (ACCPM). 
    
    Implementation applies to (inequality constrained) convex 
    optimization problems of the form
        minimize f_0(x)
        subject to f_i(x) <= 0, i = 1, ..., m,
    where f_0, ..., f_m are convex functions. The target set X is the
    epsilon-suboptimal set where epsilon is an argument of accpm.
    The ACCPM requires a set of linear inequality constraints,
    which represents a polyhedron, that all points in X satisfy. 
    That is, a matrix A and b that give constraints Ax <= b. 
    
    Parameters for convex optimization problems
    ------------
    func : callable, func(x)
        The objective function f_0 to be minimized.
    constr : tuple with callable elements
        The required format is constr = (f_1, ..., f_m) where  
        f_i(x) is callable for i = 1, ...., m, 
        So constr[i-1](x) = f_i(x) for i = 1, ..., m.
    epsilon : float
        Specifies the value to be used for the target set,
        an epsilon-suboptimal set.
    A : ndarray
        Represents the matrix A.
    b : ndarray
        Represents the vector b. 
    maxiter : int, optional
        Maximum number of iterations to perform.
    """
    k = 0
    func_values = []
    feasible_values = []
    f_best = None
    while k < maxiter + 1:
        ac = analytic_center(polyhead)
        if ac == False: 
            return False
        func_k.append(func(ac))
        if stopping() == True:
            return (True, ac, func(ac), A, b, k)
        data = oracle(ac, func, grad, constr, epsilon)
        feasibility = data[0]
        func_values.append(func(ac))
        if feasibility == True:
            feasible_values.append(func(ac))
        cp = data[1]
        (a, b) = (cp[0], cp[1])
        (A, b) = (np.vstack(A, np.transpose(a)),
                  np.vstack(b, b))
        k = k + 1